# Imports

In [1]:
import gurobipy as gp
from gurobipy import GRB, quicksum, max_
import random
random.seed(10)

# Utils

In [2]:
def gsk(dict):
    return sorted(list(dict.keys()))

# Inputs

In [3]:
slots = list(range(0, 14))
campuses = list(range(0, 3))
courses = {i: {"campus": random.choice(campuses), "is_lab": random.uniform(0, 1)>0.85} for i in range(0, 25)}
students = {200010000+i: set(random.sample(gsk(courses), random.randint(4 ,9))) for i in range(0, 200)}
venues = {i: {"campus": random.choice(campuses), "capacity": random.randint(20, 40)} for i in range(0, 30)}
print("Data Created")

Data Created


# Model

In [4]:
model = gp.Model("Midsem_Scheduling")
print("Model Created")
X = {}
for slot in slots:
    for course in gsk(courses):
        for venue in gsk(venues):
            for student in gsk(students):
                X[slot, course, venue, student] = model.addVar(vtype=GRB.BINARY, name=f'X_{slot}_{course}_{venue}_{student}')
print(f"Decision Variables Created: {len(X)}")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-04
Model Created
Decision Variables Created: 2100000


# Constraints

### 1. Each student must have exactly 1 exam for each course enrolled in, and 0 if not enrolled

In [5]:
for student in gsk(students):
    for course in gsk(courses):
        model.addConstr(quicksum(X[slot, course, venue, student] 
                                    for slot in slots
                                    for venue in gsk(venues)) == int(course in students[student]),
                        name=f'1_{student}_{course}')

### 2. Student has atmost 1 exam per slot

In [6]:
for student in gsk(students):
    for slot in slots:
        model.addConstr(quicksum(X[slot, course, venue, student] 
                                    for course in gsk(courses)
                                    for venue in gsk(venues)) <= 1,
                        name=f'2_{student}_{slot}')

### 3. Theory courses are scheduled in a single slot

In [7]:
for course in courses:
    if courses[course]["is_lab"]:
        continue
    total = gp.quicksum(X[slot, course, venue, student]
                        for venue in venues
                        for student in students
                        for slot in slots)
    for slot in slots:
        maximum = gp.max_(
            X[slot, course, venue, student]
            for venue in venues
            for student in students
        )
        sub_sum = gp.quicksum(X[slot, course, venue, student]
                              for venue in venues
                              for student in students)
        maximum_value = model.addVar(
            vtype=GRB.BINARY,
            name=f'maximum_{course}_{slot}'
        )
        model.addConstr(maximum_value == maximum, name=f'3_maximum_constraint_{course}_{slot}')
        model.addConstr(total == maximum_value * sub_sum, name=f'3_constraint_{course}_{slot}')

### 4. Total stength should not exceed venue capacity

In [8]:
for slot in slots:
    for venue in gsk(venues):
        model.addConstr(quicksum(X[slot, course, venue, student]
                                 for course in gsk(courses)
                                 for student in gsk(students)) <= venues[venue]["capacity"],
                        name=f'4_{slot}_{venue}')

### 5. Each course must be sheduled at the designated campus

In [9]:
for course in gsk(courses):
    course_campus = courses[course]["campus"]
    for venue in gsk(venues):
        venue_campus = venues[venue]["campus"]
        if course_campus != venue_campus:
            model.addConstr(quicksum(X[slot, course, venue, student]
                                        for student in gsk(students)
                                        for slot in slots) == 0,
                            name=f'5_{course}_{venue}')

### 6. Each student has atmost one exam per day

In [11]:
for student in students:
    for slot in slots:
        if slot%2==0:
            model.addConstr(quicksum(X[s, course, venue, student] 
                                    for course in gsk(courses)
                                    for venue in gsk(venues)
                                    for s in range(slot, slot+2)) <= 1,
                            name=f'6_{student}_{slot}')

# Objective